In [ ]:
%matplotlib inline

import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

np.seterr(all='ignore')

In [ ]:
# Define needed functions

def ProductionFunction(k, params):
    α, ρ, σ, δ, n, g = params
    return k**α

def InterestRate(k, params):
    α, ρ, σ, δ, n, g = params
    
    return α*k**(α-1) - δ
    
def SteadyState(params):
    α, ρ, σ, δ, n, g = params
    
    r_star = (1+n)*(1+ρ)*(1+g)**σ - 1
    k_star = (α/(r_star+δ))**(1/(1-α))
    c_star = ProductionFunction(k_star, params) - (δ+n+g+n*g)*k_star
    
    return k_star, c_star

def ResourceConstraint(k, c, params):
    α, ρ, σ, δ, n, g = params
    
    return (ProductionFunction(k, params) + (1-δ)*k - c)/((1+n)*(1+g))

def EulerEquation(k, c, params):
    α, ρ, σ, δ, n, g = params
    
    k_next = ResourceConstraint(k, c, params)
    
    if k_next > 0:
        r_next = InterestRate(k_next, params)
        c_next = ((1+r_next)/(1+ρ)/(1+n))**(1/σ) * c / (1+g)
        return c_next
    else:
        return 0

def Constant_k(k, params):
    α, ρ, σ, δ, n, g = params
    
    return ProductionFunction(k, params) - (δ+n+g+n*g)*k

In [ ]:
# Analytic check: σ=1, δ=1 case

# Parameters
α = 0.7
ρ = 1/.95-1 #0.04
σ = 1
δ = 1
n = 0 #0.01
g = 0 #0.02

params_analytic = α, ρ, σ, δ, n, g
# params_analytic = {'α':0.7, 'ρ':0.04, 'σ':1, 'δ':1, 'n':0.01, 'g':0.02}

# Steady state
print(SteadyState(params_analytic))
k_star, c_star = SteadyState(params_analytic)

In [ ]:
# Steady state
k_check = (α/((1+ρ)*(1+n)*(1+g)**σ-(1-δ)))**(1/(1-α))
c_check = Constant_k(k_check, params_analytic)
print(k_check, c_check)

In [ ]:
# Forward equations and convergence criterion

def Path(c_0, k_0, params, T=100):
    
    T += 1
    
    k_t = np.zeros(T)
    c_t = np.zeros(T)
    
    k_t[0] = k_0
    c_t[0] = c_0
    
    for t in range(T-1):
        k_t[t+1] = ResourceConstraint(k_t[t], c_t[t], params)
        if k_t[t+1] > 0:
            c_t[t+1] = EulerEquation(k_t[t], c_t[t], params)
        else:
            k_t[t+1] = 0
            c_t[t+1] = 0
            
    return k_t, c_t

def Path_crit(c_0, k_0, params, T=100):
    
    k_star, c_star = SteadyState(params)
    
    if np.sign( (k_0 - k_star) * (c_0 - Constant_k(k_0, params)) ) != 1:
        return np.inf
    
    else:
        
        k_t, c_t = Path(c_0, k_0, params, T)

        ss_diff = np.sqrt((k_t/k_star-1)**2 + (c_t/c_star-1)**2)
        
        return np.min(ss_diff)

In [ ]:
k_0_check = k_star / 1000

res = opt.minimize(Path_crit, Constant_k(k_0_check, params_analytic)/2, 
                   args=(k_0_check, params_analytic, 100), method='Powell')
print(res)

k_check, c_check = Path(res.x, k_0_check, params_analytic, T=25)

In [ ]:
plt.subplots(figsize = (5, 3))

plt.plot(k_check, c_check, 'C2.-', lw=2)
plt.plot(k_star, c_star, 'ko')
plt.show()

In [ ]:
res = opt.minimize(Path_crit, Constant_k(1.5*k_star, params_analytic)+0.01, 
                   args=(1.5*k_star, params_analytic, 100), method='Powell')
print(res)

k_check_up, c_check_up = Path(res.x, 1.5*k_star, params_analytic, T=25)

In [ ]:
plt.subplots(figsize = (5, 3))

plt.plot(k_check_up, c_check_up, 'C2.-', lw=2)
plt.plot(k_star, c_star, 'ko')
plt.show()

In [ ]:
# Number of periods
T = 25+1

k_t = np.zeros(T)
c_t = np.zeros(T)

# Solving the model forward: analytic solution
k_t[0] = k_0_check
c_t[0] = (1-α/(1+ρ)) * k_t[0]**α

for t in range(T-1):
    k_t[t+1] = (α/(1+ρ) * k_t[t]**α)/(1+n)/(1+g)
    c_t[t+1] = (1-α/(1+ρ)) * k_t[t+1]**α

# Store results for future use
k_simple, c_simple = k_t, c_t

In [ ]:
# Number of periods
T = 25+1

k_t = np.zeros(T)
c_t = np.zeros(T)

# Solving the model forward: analytic solution
k_t[0] = 1.5*k_star
c_t[0] = (1-α/(1+ρ)) * k_t[0]**α

for t in range(T-1):
    k_t[t+1] = (α/(1+ρ) * k_t[t]**α)/(1+n)/(1+g)
    c_t[t+1] = (1-α/(1+ρ)) * k_t[t+1]**α

# Store results for future use
k_simple_up, c_simple_up = k_t, c_t

In [ ]:
# Convergence plot
plt.subplots(figsize = (5, 3))

plt.hlines(k_star, 0, T-1, color='C3', lw=1, linestyle='--') #, label='$k^{*}$'
plt.hlines(c_star, 0, T-1, color='C0', lw=1, linestyle='--') #, label='$c^{*}$'

plt.plot(k_simple, color='C3', lw=2, label='$k$')
plt.plot(c_simple, color='C0', lw=2, label='$c$')

plt.ylim(0, 0.3)
# plt.yticks([k_0, c_star, k_star], ['$k_{0}$', '$c^{*}$', '$k^{*}$'])
plt.yticks([c_star, k_star], ['$c^{*}$', '$k^{*}$'])

# plt.title('Convergence to steady state over time')
# plt.xlabel('Period')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Phase diagram
fig, ax = plt.subplots(figsize = (5, 3))

kk = np.linspace(0, 1.5*k_star, 1000)
cc = np.linspace(0, 1.8*c_star, 1000)

plt.hlines(c_star, 0, k_star, 'C0', lw=1, linestyle='--')

plt.plot(kk, (1-α/(1+ρ))*kk**α, 'C2', lw=2)
plt.plot(kk, Constant_k(kk, params_analytic), 'C0', lw=2, label='$k_{t+1}=k_{t}$')
plt.plot(kk**0 * k_star, cc, 'C3', lw=2, label='$c_{t+1}=c_{t}$')
plt.plot(kk[-1], (1-α/(1+ρ))*kk[-1]**α, 'C2', lw=2, label='Ścieżka przejścia')
plt.plot(k_star, c_star, 'ko', label='Stan ustalony')

# plt.plot(kk, (1-α*β)*kk**α, 'C2', lw=2, label='Saddle path')
# plt.plot(k_star, c_star, 'ko', label='Steady state')

# plt.title('Phase diagram of the RCK model')
# plt.xlabel('Capital per worker $k$')
# plt.ylabel('Consumption per worker $c$')
# plt.legend(loc='lower right')
plt.legend(loc='upper left')

plt.xlim(0, kk[-1])
plt.ylim(0, cc[-1])

# plt.xticks([0, k_star, k_GR, kk[-1]], [0, '$k^{*}$', '$k^{*}_{GR}$', '$k$'])
plt.xticks([0, k_star, kk[-1]], [0, '$k^{*}$', '$k$'])
plt.yticks([c_star, cc[-1]], ['$c^{*}$', '$c$'])

plt.show()

In [ ]:
# Phase diagram
fig, ax = plt.subplots(figsize = (5, 3))

kk = np.linspace(0, 1.5*k_star, 1000)
cc = np.linspace(0, 1.8*c_star, 1000)

k_GR = (α/(δ+n+g+n*g))**(1/(1-α))

plt.hlines(c_star, 0, k_star, 'C0', lw=1, linestyle='--')
plt.vlines(k_GR, 0, Constant_k(k_GR, params_analytic), 'C6', lw=1, linestyle='--')

plt.plot(k_check, c_check, 'C2', lw=3)
plt.plot(k_simple, c_simple, 'k--', lw=1)

plt.plot(kk, Constant_k(kk, params_analytic), 'C0', lw=2, label='$k_{t+1}=k_{t}$')
plt.plot(kk**0 * k_star, cc, 'C3', lw=2, label='$c_{t+1}=c_{t}$')

plt.plot(k_check_up, c_check_up, 'C2', lw=3, label='Ścieżka przejścia (numeryczna)')
plt.plot(k_simple_up, c_simple_up, 'k--', lw=1, label='Ścieżka przejścia (analityczna)')

plt.plot(k_star, c_star, 'ko') #, label='Stan ustalony'

# plt.title('Phase diagram of the RCK model')
# plt.xlabel('Capital per worker $k$')
# plt.ylabel('Consumption per worker $c$')
# plt.legend(loc='lower right')
plt.legend(loc='upper left')

plt.xlim(0, kk[-1])
plt.ylim(0, cc[-1])

plt.xticks([0, k_star, k_GR, kk[-1]], [0, '$k^{*}$', '$k^{*}_{GR}$', '$k$'])
plt.yticks([c_star, cc[-1]], ['$c^{*}$', '$c$'])

plt.show()

In [ ]:
# Regular case

# Parameters
α = 0.33
ρ = 0.04
σ = 2
δ = 0.1
n = 0.01
g = 0*0.02

params = α, ρ, σ, δ, n, g

# #           α, ρ, σ, δ, n, g
# params = 0.33, 0.04, 2, 0.1, 0.01, 0.0

# Steady state
print(SteadyState(params))
k_star, c_star = SteadyState(params)

In [ ]:
# Find the function minimum, starting from an initial guess
k_0 = k_star / 200

result = opt.minimize(Path_crit, Constant_k(k_0, params)/2, 
                      args=(k_0, params, 100), method='Powell')
print(result)

c_0 = result.x

In [ ]:
# Find the function minimum, starting from an initial guess
result = opt.minimize(Path_crit, Constant_k(2*k_star, params)+0.01, 
                      args=(2*k_star, params, 100), method='Powell')
print(result)

c_0_up = result.x

In [ ]:
kk_g, cc_g = np.meshgrid(kk, cc)

def Δk(k, c):
    return k**α-(δ+n+g+n*g)*k-c

def Δc(k, c):
    return (((1+α*k**(α-1)-δ)/(1+ρ)/(1+n))**(1/σ)/(1+g)-1)*c

fig, ax = plt.subplots(figsize = (5, 3))

plt.plot(k_t, c_t, 'C2', lw=2)
plt.plot(kk, Constant_k(kk, params), 'C0', lw=2, label='$k_{t+1}=k_{t}$')
plt.plot(kk**0 * k_star, cc, 'C3', lw=2, label='$c_{t+1}=c_{t}$')
plt.plot(k_t_up, c_t_up, 'C2', lw=2, label='Ścieżka przejścia')
plt.plot(k_star, c_star, 'ko', label='Stan ustalony')

plt.xlim(0, kk[-1])
plt.ylim(0, cc[-1])

plt.xticks([0, k_star, kk[-1]], [0, '$k^{*}$', '$k$'])
plt.yticks([c_star, cc[-1]], ['$c^{*}$', '$c$'])

plt.legend(loc='upper left')        #, frameon=True

plt.show()

In [ ]:
s_t = 1 - c_t/ProductionFunction(k_t, params)
s_star = 1-c_star/ProductionFunction(k_star, params)

s_t = np.append(s_t, s_t[-1])
s_t = np.append(s_t, s_t[-1])
s_t = np.append(s_t, s_t[-1])
s_t = np.append(s_t, s_t[-1])

T = 100

plt.subplots(figsize = (3, 3))
# plt.plot(k_t[3:104], lw=2)
plt.plot(k_t, lw=2)
plt.hlines(k_star, 0, T, lw=1, linestyle='--')
plt.title(r'Capital per effective labor $\hat{k}$')
plt.xlabel('Period')
plt.show()

plt.subplots(figsize = (3, 3))
# plt.plot(c_t[3:104], lw=2)
plt.plot(c_t, lw=2)
plt.hlines(c_star, 0, T, lw=1, linestyle='--')
plt.title(r'Consumption per effective labor $\hat{c}$')
plt.xlabel('Period')
plt.show()

plt.subplots(figsize = (3, 3))
plt.plot(100*s_t[3:104], lw=2)
# plt.plot(100*s_t, lw=2)
plt.hlines(100*s_star, 0, T, lw=1, linestyle='--')
plt.title('Saving rate $s$ (%)')
plt.xlabel('Period')
# plt.ylim(20, 30)
plt.show()

In [ ]:
def EulerEquation_SG(k, c, params, c_min=0):
    α, ρ, σ, δ, n, g = params
    
    k_next = ResourceConstraint(k, c, params)
    
    if k_next > 0:
        r = InterestRate(k_next, params)
        c_next = ((1+r)/(1+n)/(1+ρ))**(1/σ) * (c-c_min) / (1+g) + c_min
        return c_next
    else:
        return 0

In [ ]:
# Forward equations and convergence criterion

def Path_SG(c_0, k_0, params, c_min=0, T=100):
    
    T += 1
    
    k_t = np.zeros(T)
    c_t = np.zeros(T)
    
    k_t[0] = k_0
    c_t[0] = c_0
    
    for t in range(T-1):
        k_t[t+1] = ResourceConstraint(k_t[t], c_t[t], params)
        if k_t[t+1] > 0:
            c_t[t+1] = EulerEquation_SG(k_t[t], c_t[t], params, c_min)
        else:
            k_t[t+1] = 0
            c_t[t+1] = 0
            
    return k_t, c_t

def Path_crit_SG(c_0, k_0, params, c_min=0, T=100):
    
    k_star, c_star = SteadyState(params)
    
    if np.sign( (k_0 - k_star) * (c_0 - Constant_k(k_0, params)) ) != 1:
        return np.inf
    
    else:
        
        k_t, c_t = Path_SG(c_0, k_0, params, c_min, T)

        ss_diff = np.sqrt((k_t/k_star-1)**2 + (c_t/c_star-1)**2)
        
        return np.min(ss_diff)

In [ ]:
# Parameters for Stone-Geary
#           α, ρ, σ, δ, n, g
params_SG = 0.33, 0.04, 2, 0.1, 0.01, 0.0
c_min = 0.5

# Steady state
print(SteadyState(params_SG))
k_star_SG, c_star_SG = SteadyState(params_SG)

# Initial level of capital
k_0_SG = k_star_SG / 20
print(Constant_k(k_0_SG, params_SG))
Constant_k(k_0_SG, params_SG) > c_min
# k_0_SG

In [ ]:
# Find the function minimum, starting from an initial guess
result_SG = opt.minimize(Path_crit_SG, c_min,
                         args=(k_0_SG, params_SG, c_min, 100), method='Powell')
print(result_SG)

c_0_SG = result_SG.x

In [ ]:
k_t_SG, c_t_SG = Path_SG(c_0_SG, k_0_SG, params_SG, c_min, 100)

y_t_SG = ProductionFunction(k_t_SG, params_SG)

s_t_SG = 1 - c_t_SG/y_t_SG

g_y_t_SG = 100*(y_t_SG[1:]/y_t_SG[:-1]-1)

In [ ]:
plt.plot(k_t_SG, lw=2)
plt.hlines(k_star_SG, 0, 100, lw=1, linestyle='--')
plt.title('Capital per person $k$')
plt.xlabel('Period')
plt.show()

plt.plot(c_t_SG, lw=2)
plt.hlines(c_star_SG, 0, 100, lw=1, linestyle='--')
plt.title('Consumption per person $c$')
plt.xlabel('Period')
plt.show()

plt.plot(100*s_t_SG, lw=2)
plt.hlines(100*s_t_SG[-1], 0, 100, lw=1, linestyle='--')
plt.title('Saving rate $s$ (%)')
plt.xlabel('Period')
plt.show()

plt.plot(g_y_t_SG, lw=2)
plt.title('Growth rate of GDP per person $g_{y}$ (%)')
plt.xlabel('Period')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (5, 3))
plt.plot(np.log(1+y_t_SG[:-20]), 100*s_t_SG[:-20], lw=2)
plt.title('Saving rate vs income for Stone-Geary utility function')
plt.xlabel('Logarithm of GDP per person')
plt.ylabel('Saving rate (%)')
plt.show()

fig, ax = plt.subplots(figsize = (5, 3))
plt.plot(np.log(1+y_t_SG[:-1]), g_y_t_SG, lw=2)
plt.title('Growth rate vs income for Stone-Geary utility function')
plt.xlabel('Logarithm of GDP per person')
plt.ylabel('Annual growth rate (%)')
plt.show()

In [ ]:
# Find the function minimum, starting from an initial guess
result = opt.minimize(Path_crit, c_min,
                      args=(k_0_SG, params_SG, 100), method='Powell')
print(result)

c_0 = result.x

In [ ]:
k_t, c_t = Path(c_0, k_0_SG, params_SG, 100)

y_t = ProductionFunction(k_t, params_SG)

s_t = 1 - c_t/y_t

g_y_t = 100*(y_t[1:]/y_t[:-1]-1)

In [ ]:
# T = 50

plt.subplots(figsize = (3, 3))
plt.hlines(k_star_SG, 0, T, lw=1, linestyle='--')
plt.plot(k_t, lw=2, label='CRRA')
plt.plot(k_t_SG, lw=2, label='Stone-Geary')
plt.title('k')
plt.legend(loc='lower right')
plt.show()

plt.subplots(figsize = (3, 3))
plt.hlines(c_star_SG, 0, T, lw=1, linestyle='--')
plt.plot(c_t, lw=2, label='CRRA')
plt.plot(c_t_SG, lw=2, label='Stone-Geary')
plt.title('c')
plt.legend(loc='lower right')
plt.show()

plt.subplots(figsize = (3, 3))
plt.hlines(100*s_t_SG[-1], 0, T, lw=1, linestyle='--')
plt.plot(100*s_t[3:104], lw=2, label='CRRA')
plt.plot(100*s_t_SG, lw=2, label='Stone-Geary')
plt.title('s')
plt.legend(loc='lower right')
plt.show()

plt.subplots(figsize = (3, 3))
# plt.plot(g_y_t_SG, lw=2)
# plt.hlines(100*s_t_SG[-1], 0, T, lw=1, linestyle='--')
plt.plot(g_y_t[:51], lw=2, label='CRRA')
plt.plot(g_y_t_SG[:51], lw=2, label='Stone-Geary')
plt.title('g_y')
plt.legend(loc='upper right')
plt.ylim(0, 8)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (5, 3))
plt.plot(np.log(1+y_t[:-20]), 100*s_t[:-20], lw=2, label='CRRA')
plt.plot(np.log(1+y_t_SG[:-20]), 100*s_t_SG[:-20], lw=2, label='Stone-Geary')
# plt.title('Saving rate vs income for Stone-Geary utility function')
plt.xlabel('Logarithm of GDP per person')
plt.ylabel('Saving rate (%)')
plt.legend(loc='lower right')
plt.xlim(0.4, 0.9)
plt.show()

fig, ax = plt.subplots(figsize = (5, 3))
plt.plot(np.log(1+y_t[:-1]), g_y_t, lw=2, label='CRRA')
plt.plot(np.log(1+y_t_SG[:-1]), g_y_t_SG, lw=2, label='Stone-Geary')
# plt.title('Growth rate vs income for Stone-Geary utility function')
plt.xlabel('Logarithm of GDP per person')
plt.ylabel('Annual growth rate (%)')
plt.legend()
plt.xlim(0.4, 0.9)
plt.ylim(0, 8)
plt.show()

In [ ]:
# Phase diagram (realistic params)
fig, ax = plt.subplots(figsize = (5, 3))

# k_t, c_t = Path_SG(c_0, k_0, params, 100)
# k_t_up, c_t_up = Path(c_0_up, 2*k_star, params, 100)

kk = np.linspace(0, 1.5*k_star_SG, 1000)
cc = np.linspace(0, 1.5*c_star_SG, 1000)

plt.hlines(c_star_SG, 0, k_star_SG, 'C0', lw=1, linestyle='--')
plt.hlines(c_min, 0, k_star_SG, 'C1', lw=1, linestyle='--')
# plt.vlines(k_GR, 0, A*k_GR**α - (n+δ)*k_GR, 'C6', lw=1, linestyle='--')

plt.plot(k_t_SG, c_t_SG, 'C1', lw=2, label='Ścieżka przejścia (Stone-Geary)')
plt.plot(k_t, c_t, 'C2', lw=2, label='Ścieżka przejścia (CRRA)')

plt.plot(kk, Constant_k(kk, params_SG), 'C0', lw=2) #, label='$k_{t+1}=k_{t}$')
plt.plot(kk**0 * k_star_SG, cc, 'C3', lw=2) #, label='$c_{t+1}=c_{t}$')
plt.plot(k_star_SG, c_star_SG, 'ko') #, label='Stan ustalony')

# plt.plot(kk, (1-α*β)*kk**α, 'C2', lw=2, label='Saddle path')
# plt.plot(k_star, c_star, 'ko', label='Steady state')

# plt.title('Phase diagram of the RCK model')
# plt.xlabel('Capital per worker $k$')
# plt.ylabel('Consumption per worker $c$')
# plt.legend(loc='lower right')
plt.legend(loc='upper left')

plt.xlim(0, kk[-1])
plt.ylim(0, cc[-1])

plt.xticks([0, k_star_SG, kk[-1]], [0, '$k^{*}$', '$k$'])
plt.yticks([c_min, c_star_SG, cc[-1]], [r'$\bar{c}$', '$c^{*}$', '$c$'])

plt.show()